In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import random
random.seed(42)
import math

In [2]:
zone = 0
input_folder = f"6_all_data_table/zone_{zone}"

# Files per category
categories = [3, 4, 5]
category_c_files = {key:[] for key in categories}
for filename in os.listdir(input_folder):
    category = int(filename[1])
    category_c_files[category].append(filename)

In [3]:
columns = ['valid_time', 'time_step', 'latitude', 'latitude_rank',
              'longitude', 'longitude_rank', 'coord_index', 'u10', 'u100', 'v10', 'v100',
              'w10', 'w100', 'tp', 't2m', 'lsm', 'sp', 'tcw', 'e']

quantile_measure = 'w10'
print(f"Quantile measure used: {quantile_measure}")

Quantile measure used: w10


In [4]:
def reservoir_sample_L_from_files(file_list, column, k, verbose=False):
    """
    Implements Algorithm L for reservoir sampling across multiple files.
    
    :param file_list: List of file paths to process
    :param column: Name of the column to sample from
    :param k: Number of samples to retain
    :return: List containing the final k samples
    """
    reservoir = []  # Reservoir to store the selected sample
    total_count = 0  # Tracks how many items have been seen
    total_count_at_file_end = -1
    next_file_count = 0
    go_to_next = 0

    W = math.exp(math.log(random.random()) / k) 
    # Maximum of k uniform values in [0, 1] (first skipping weight)

    for file in file_list:
        if verbose:
            print(f"Processing file: {file}")
        df = pd.read_csv(file, usecols=[column])  # Read only the necessary column
        current_file_length = len(df)
        total_count_at_file_start = total_count_at_file_end + 1
        total_count_at_file_end = total_count_at_file_start + current_file_length - 1

        # First k values
        while len(reservoir) < k:
            if total_count <= total_count_at_file_end:
                reservoir.append(df[column][total_count - total_count_at_file_start])
                total_count += 1
            else:
                next_file_count = total_count - total_count_at_file_end
                go_to_next = 1
                break
        if go_to_next:
            continue

        if next_file_count:
            if next_file_count<current_file_length:
                reservoir[random.randint(0, k - 1)] = df[column][next_file_count-1]
            else:
                next_file_count = next_file_count - current_file_length
                go_to_next = 1
        if go_to_next:
            continue

        while True:
            skip = math.floor(math.log(random.random()) / math.log(1 - W)) + 1
            # Skip from geometric distribution with probability of success W
            W *= math.exp(math.log(random.random()) / k)
            # Maximum of k uniform values in [0, W] (next skipping weight)
            next_count = total_count + skip
            if next_count <= total_count_at_file_end:
                total_count = next_count
                reservoir[random.randint(0, k - 1)] = df[column][total_count - total_count_at_file_start]
            else:
                total_count = next_count
                next_file_count = total_count - total_count_at_file_end
                break

    return reservoir  # Return the final sampled values

In [11]:
random.seed(42)
# Reservoir samples
L_samples = {}
for c in categories:
    files = [os.path.join(input_folder, filename) for filename in category_c_files[c]]
    L_samples[c] = reservoir_sample_L_from_files(files, 'w10', 100000)
    print(f"Category {c} finished. Reservoir sample length {len(L_samples[c])}")
    print(f"95% quantile {np.quantile(np.array(L_samples[c]),0.95):.3f}")

Category 3 finished. Reservoir sample length 100000
95% quantile 11.763
Category 4 finished. Reservoir sample length 100000
95% quantile 12.215
Category 5 finished. Reservoir sample length 100000
95% quantile 12.547


In [12]:
# Find quantiles with reservoir sampling
quantiles = [.50, .80, .90, .95, .95, .99, .995]
category_c_quantiles = {}
for c in categories:
    sample = L_samples[c]
    category_c_quantiles[c] = {q : np.quantile(sample, q) for q in quantiles}

print(f"Zone {zone}")
for c in categories:
    print(f"\n Category {c}")
    q_c = category_c_quantiles[c]
    for q, qv in q_c.items():
        print(f"{100*q:.1f}% : {qv:.2f}")

Zone 0

 Category 3
50.0% : 4.99
80.0% : 7.77
90.0% : 9.65
95.0% : 11.76
99.0% : 17.25
99.5% : 19.49

 Category 4
50.0% : 5.18
80.0% : 8.30
90.0% : 10.24
95.0% : 12.22
99.0% : 17.43
99.5% : 19.87

 Category 5
50.0% : 4.94
80.0% : 7.87
90.0% : 10.13
95.0% : 12.55
99.0% : 18.83
99.5% : 21.50


In [17]:
total = 0
kept = 0
quantile = 0.95
quantile_str = "(q" + str(int(quantile*100)) + ")"
for c in categories:
    quantile_val = np.quantile(np.array(L_samples[c]), quantile)
    print(f"Category {c} {quantile} quantile value {quantile_val}")
    files = [os.path.join(input_folder, filename) for filename in category_c_files[c]]
    for filename in files:
        new_name = "8_filtered_data_table" + filename[16:-4] + quantile_str + ".csv"

        # Find rows
        df = pd.read_csv(filename, index_col=False)
        total += len(df)
        df = df.loc[df[quantile_measure]>=quantile_val,:]
        kept += len(df)
        df.to_csv(new_name, index=False)
    print(f"Finished processing category {c} \n Kept {kept} out of {total} rows ({kept/total*100:.2f}) \n")

Category 3 0.95 quantile value 11.76308688205934
Finished processing category 3 
 Kept 887173 out of 17705988 rows (5.01) 

Category 4 0.95 quantile value 12.215182456939443
Finished processing category 4 
 Kept 1980483 out of 39498228 rows (5.01) 

Category 5 0.95 quantile value 12.546528792597863
Finished processing category 5 
 Kept 2395110 out of 47808828 rows (5.01) 

